In [ ]:
import torch

'''
1. 设置随机数种子
2. 设备
3. 其他参数设置
4. 数据集
5. 模型
6. 定义损失函数和优化器
7. 训练和测试

损失Loss为Nan/inf或者超级大的原因:
	训练或者预测过程中经常会遇到训练损失值或者验证损失值不正常, 无穷大, 或者直接nan的情况
	可能原因：
		1. 梯度爆炸造成Loss爆炸 --> 降低初始的学习率, 并设置学习率衰减
		2. 损失函数可能不正确 --> 考虑在除数中加入微小的常数保证计算稳定性
			注意除以0和log0的情况 --> log(x+1e-6)
		3. 输入不要引入nan --> 数据要normalized
		4. batchNorm可能捣鬼, 这种情况很有可能发生在预测阶段, 即使用model.eval()之后 --> 如果你在预测阶段也将模型model设置为model.train(True), 那么问题可能就不会出现; 或者设置Batchnorm中的参数track_running_stats=False使移动均值和移动方差不起作用
		5. 你的Shuffle设置有没有乱动, 一般我们是在训练阶段开启shuffle而在预测阶段关闭shuffle, 但是假如我们使用了batch_norm层, 并且数据的分布极不规律(使用shuflle和不使用shuffle读取的数据顺序的信息分布完全不同), 那么在训练阶段训练好的模型(使用shuffle), 在预测阶段使用的时候(不使用shuffle), 由于数据分布的不同, 也是有可能导致batch_norm层出现nan, 从而导致不正常的损失函数出现

训练策略: 
	1. 加速模型运行 [https://efficientdl.com/faster-deep-learning-in-pytorch-a-guide/]
		1.1 调大batch_size以及学习率
			在通常情况下, 使用GPU内存允许的最大批处理量可以加快训练速度; 一般来说, 将批量大小增加一倍, 学习率也提高一倍
			增加GPU的内存占用率, 尽量用完内存, 而不要剩一半; 空的内存给另外的程序用, 两个任务的效率都会非常低
			使用大batch的不足是, 这可能导致解决方案的泛化能力比使用小batch的差
			调小batch_size, 运行速度慢
		1.2 选择合适的学习率时间表
			选择的学习率时间表对收敛速度以及模型的泛化性能有很大影响
			定期增加学习率有助于更快地穿越损失函数中的鞍点
			如torch.optim.lr_scheduler.CyclicLR和torch.optim.lr_scheduler.OneCycleLR
			1Cycle包括两个等长的步幅, 一个步幅是从较低的学习率到较高的学习率, 另一个是回到最低水平
			最大值来自学习率查找器选取的值, 较小的值可以低十倍; 然后这个周期的长度应该略小于总的epochs数
			并且, 在训练的最后阶段, 我们应该允许学习率比最小值小几个数量级
			与传统的学习率schedule相比, 在最好的情况下, 该schedule实现了巨大的加速
		1.3 设置dataloader的num_worker(4或8或16)和pin_memory(页锁定内存)=True以及non_blocking=True
			但是有时候即使增大num_worker也无法提高GPU利用率, 这是因为训练的瓶颈在IO
			num_worker经验值为cpu核心数或gpu的数量或gpu的数量的4倍
			worker数量的多和少都会导致速度变慢, 数量越多还会增加CPU内存消耗
			.cuda(non_blocking=True)
			数据预处理尽量不要放在loader中做
		1.4 使用不同的优化器
			比如AdamW, AdamW是带有权重衰减(而不是L2正则化)的Adam, 它在错误实现, 训练时间都胜过Adam(在误差和训练时间上都一直优于Adam)
			Adam和AdamW都能与上面提到的1Cycle策略很好地搭配
			此外, 还有一些非本地的优化器值得关注, 比如, LARS和LAMB
		1.5 打开cudNN基准测试
			如果模型架构保持固定, 输入大小保持不变, 则可以设置torch.backends.cudnn.benchmark = True
			启动cudNN自动调整器, 它将对cudNN中计算卷积的多种不同方法进行基准测试, 以获得最佳的性能指标
		1.6 防止CPU和GPU之间频繁传输数据
			注意经常使用tensor.cpu()将张量从GPU传输到CPU(或使用tensor.cuda()将张量从CPU转到GPU)
			.item()和.numpy()也是一样可以使用.detach()代替
			如果正在创建一个张量, 可以使用关键字参数device=torch.device('cuda:0')直接将其分配给对应的GPU
			如果到传输数据的情境下, 可以使用.to(non_blocking=True), 只要在传输后没有任何同步点
		1.7 使用.as_tensor()而不是.tensor()
			torch.tensor()总是会复制数据
			如果要转换一个numpy数组, 使用torch.as_tensor()或torch.from_numpy()来避免复制数据
		1.8 梯度裁剪
			对RNNs, 基于Transformer和ResNets的架构以及一系列不同的优化器都非常有用
		1.9 在BatchNorm之前关闭偏置
			在开始Batch Normalization层之前关闭bias层
			对于一个2-D卷积层, 可以将bias关键字设置为False: torch.nn.Conv2d(..., bias=False, ...)
		1.10 使用输入和batch归一化
			要再三检查一下输入是否归一化, 是否使用了batch归一化
		1.11 在验证过程中关闭梯度计算
			在验证期间设置torch.no_grad()
		1.12 使用梯度/激活检查点checkpointing
			检查点的工作原理, 是用计算换取内存; 并不存储整个计算图的所有中间激活用于backward pass
			而是重新计算这些激活, 我们可以将其应用于模型的任何部分
			具体来说, 在前向传递中, 函数会以torch.no_grad()方式运行, 不存储中间激活
			相反, 前向传递会保存输入元组以及函数参数
			在后向传递中, 输入和函数会被检索, 并再次对函数进行前向传递计算
			然后跟踪中间激活, 使用这些激活值计算梯度。
			虽然这可能会略微增加给定batch大小的运行时间, 但会显著减少内存占用
			这反过来又将允许进一步增加所使用的batch大小, 从而提高GPU的利用率
		1.13 使用自动混合精度(AMP)
		1.14 使用梯度积累
			增加batch大小的另一种方法是在调用optimizer.step()之前在多个.backward()传递中累积梯度
			这个方法主要是为了规避GPU内存限制而开发的
		1.15 使用分布式数据并行进行多GPU训练
			加速分布式训练可能有很多方法, 但是简单的方法是使用torch.nn.DistributedDataParallel而不是torch.nn.DataParallel
			这样一来, 每个GPU将由一个专用的CPU核心驱动, 避免了DataParallel的GIL问题
			[https://pytorch.org/tutorials/beginner/dist_overview.html]
		1.16 设置梯度为None而不是0
			梯度设置为.zero_grad(set_to_none=True)而不是.zero_grad()
			这样做可以让内存分配器处理梯度, 而不是将它们设置为0
			正如文档中所说, 将梯度设置为None会产生适度的加速, 但不要期待奇迹出现
			注意, 这样做也有缺点, 详细信息请查看文档[https://pytorch.org/docs/stable/optim.html]
		1.17 必要时打开调试工具
			PyTorch提供了很多调试工具
			例如autograd.profiler, autograd.grad_check, autograd.anomaly_detection
			请确保当你需要调试时再打开调试器, 不需要时要及时关掉, 因为调试器会降低你的训练速度
		1.18 数据变换(用于数据增强))可成为速度提升的另一个来源
			一些只使用简单Python语句的变换可以通过使用numba包来加速
		1.19 将数据集预处理成单个文件, 对速度也有好处
		
	2. 使模型较快收敛
		2.1 增大学习率
		2.2 使用adam这类优化器
		2.3 Rnn的梯度裁剪
'''

In [ ]:
# 设置随机数种子
import numpy as np
import random
import os
seed = 42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) # 为当前的GPU设置产生随机数的种子
# torch.cuda.manual_seed_all(seed) # 为所有GPU设置产生随机数的种子
torch.backends.cudnn.deterministic = True # 每次返回的卷积算法将是确定的; 会降低训练速度; 从checkpoints重新开始时会出现意外的结果
# torch.backends.cudnn.benchmark = False # 设置为True, 可以大大提升卷积神经网络的运行速度; 可在网络训练开始前设置

# torch.manual_seed(4) 设置生成随机数的种子, 返回一个torch._C.Generator对象, 如<torch._C.Generator object at 0x0000019684586350>
torch.initial_seed() # 返回生成随机数的原始种子值; 4

In [ ]:
# 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 后续的数据以及模型需要.to(device)
torch.cuda.set_device(0) # model = model.cuda(); model = model.cuda(0)

torch.cuda.device_count()
torch.cuda.get_device_name()

In [ ]:
# 其他参数设置: 包括数据集, 模型, 训练等参数

In [ ]:
# 数据集

torch.utils.data.DataLoader()

In [ ]:
# 模型

# 预训练模型
from torch import nn
# 若仅想在模型顶层微调, 则按如下设置:
pretrain_model = None
for param in pretrain_model.parameters():
    param.requires_grad = False
pretrain_model.fc = nn.Linear(pretrain_model.fc.in_features, 100) # 替换顶层来微调

# 下载模型
# 下载整个模型
model = torch.load('model.ckpt', map_location=torch.device('cuda:0'))
# map_location指明此时的device环境; map_location='cpu'
# 如果保存的模型是torch.nn.DataParallel, 则当前的模型也需要是
# model.load_state_dict(checkpoint['state_dict']) 只下载模型参数; model.load_state_dict(torch.load('params.ckpt'))
	# 函数load_state_dict有参数strict, 默认strict=True; 判断参数拷贝过程中是否有unexpected_keys或者missing_keys, 如果有就报错, 代码不能继续执行
	# 如果strict=False, 则会忽略这些细节
	# model.state_dict()返回的是一个OrderDict, 存储了网络结构的名字和对应的参数
	# del state_dict
    # torch.cuda.empty_cache()
# optimizer.load_state_dict(checkpoint['optimizer'])

# 分布式模型
model = torch.nn.DataParallel(model).cuda()
torch.nn.DataParallel(model, device_ids=[0,1,2], output_device=None, dim=0) # dim只能取值-1或0
# 首先在前向过程中, 输入数据会被划分成多个子部分(以下称为副本)送到不同的device中进行计算, 模型是在每个device上复制一份
# 输入的batch是会被平均分到每个device中去, 但是模型是要拷贝到每个devide中去的, 每个模型只需要处理每个副本即可
# 要保证batch size大于gpu个数
# 然后在反向传播过程中, 每个副本的梯度被累加到原始模块中
# 概括来说就是: DataParallel会自动帮我们将数据切分到相应GPU, 将模型复制到相应GPU, 进行正向传播计算梯度并汇总

# 计算模型整体参数量
num_parameters = sum(torch.numel(parameter) for parameter in model.parameters())
sum([p.data.nelement() for p in model.parameters()])
# 可训练参数
sum(p.numel() for p in model.parameters() if p.requires_grad)
net_parameters = filter(lambda p: p.requires_grad, model.parameters())
sum([np.prod(p.size()) for p in net_parameters])
total_params = 0
for x in filter(lambda p: p.requires_grad, model.parameters()):
	total_params += np.prod(x.data.numpy().shape)
# 计算网络总层数
len(list(filter(lambda p: p.requires_grad and len(p.data.size()) > 1, model.parameters())))

In [ ]:
# 定义损失函数和优化器

# 优化器: 计算导数的算法
optimizer = torch.optim.SGD() 
# 第一个参数可取model.parameters()
# 还有参数lr, weight_decay, momentum
# 梯度下降
	# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
	# linear.bias.data.sub_(0.01 * linear.bias.grad.data)
torch.optim.Adam()
# 第一个参数可取model.parameters()
# 还有参数lr, weight_decay, betas, eps, amsgrad
# Adagrad效果比Adam好的多
torch.optim.AdamW()
# 第一个参数可取model.parameters()
# 还有参数lr, weight_decay, eps
torch.optim.Adadelta()

# 只有叶子张量(在图的开始被创建, 即不是通过图中的操作产生的)能被优化
# 当对requires_grad=True的张量应用任意操作时, 这些操作会被追踪以便后续进行反向传播
# 无法将上述的任意中间结果输入优化器

# 可根据epoch修改学习率
'''
for param_group in optimizer.param_groups:  
	param_group['lr'] = lr
'''

# 利用scheduler调整学习率
lr_milestones = []
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_milestones, gamma=0.1, last_epoch=-1, verbose=False)
# 当训练epoch达到milestones值时, 初始学习率乘以gamma得到新的学习率
scheduler.step() # 在训练迭代中进行, 且应使得optimizer.step()置于其前
float(scheduler.get_lr()[0])
scheduler.get_last_lr()

lr_decay_step = 1
torch.optim.lr_scheduler.StepLR(optimizer, lr_decay_step, gamma=0.999975)

torch.optim.lr_scheduler.CyclicLR()
torch.optim.lr_scheduler.OneCycleLR()

In [ ]:
# 训练和测试

# 训练
model.train()
# 迭代进行(例如[for images, labels in train_loader]); 若只想显示最终的结果, 则可借助IPython的display.clear_output(wait=True), 置于输出语句之后
'''
1. 前向传播
2. 计算损失loss
3. 将梯度置零: optimizer.zero_grad()
    也可置于最前面
    model.zero_grad()会把整个模型的参数的梯度都归零
    optimizer.zero_grad()只会把传入其中的参数的梯度归零
    loss.backward()前用optimizer.zero_grad()清除累积梯度
4. 反向传播: loss.backward() -> 计算梯度
5. 更新参数: optimizer.step() -> 一步梯度下降
    在反向传播之后更新参数之前, 可进行梯度裁剪: nn.utils.clip_grad_norm_()
'''

# 测试
model.eval()
# 匹配with torch.no_grad()使用

# 可绘图(针对二维数据)
# X是二维数据, y是标签
from matplotlib import pyplot as plt
def plot_data(X, y, auto=False, zoom=1):
    X = X.cpu()
    y = y.cpu()
    plt.scatter(X.numpy()[:, 0], X.numpy()[:, 1], c=y, s=20, cmap=plt.cm.Spectral)
    plt.axis('square')
    plt.axis(np.array((-1.1, 1.1, -1.1, 1.1)) * zoom)
    if auto is True: plt.axis('equal')
    plt.axis('off')

    _m, _c = 0, '.15'
    plt.axvline(0, ymin=_m, color=_c, lw=1, zorder=0)
    plt.axhline(0, xmin=_m, color=_c, lw=1, zorder=0)
def plot_model(X, y, model):
    model.cpu()
    mesh = np.arange(-1.1, 1.1, 0.01)
    xx, yy = np.meshgrid(mesh, mesh)
    with torch.no_grad():
        data = torch.from_numpy(np.vstack((xx.reshape(-1), yy.reshape(-1))).T).float()
        Z = model(data).detach()
    Z = np.argmax(Z, axis=1).reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.3)
    plot_data(X, y)

# 存储模型
# 存储整个模型
torch.save(model, 'model.ckpt') 
# 只保存模型参数
torch.save(model.state_dict(), 'params.ckpt')
# 可保存自定义字典, 文件名如ckpt.pth.tar
	# 一般包括模型, 优化器, 迭代轮次epoch(便于中断后恢复训练后), 最佳acc
import shutil
start_epoch = 0
resume = 0
num_epochs = None
current_acc = None
if resume: # resume为参数, 第一次训练时设为0, 中断再训练时设为1
    model_path = os.path.join('model', 'best_checkpoint.pth.tar')
    assert os.path.isfile(model_path)
    checkpoint = torch.load(model_path)
    best_acc = checkpoint['best_acc']
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print('Load checkpoint at epoch {}.'.format(start_epoch))
    print('Best accuracy so far {}.'.format(best_acc))

for epoch in range(start_epoch, num_epochs): 
    ... 

    # Test the model
    ...

    # save checkpoint(若best_acc一直没更新, 也可设置早停止或者减小lr)
    is_best = current_acc > best_acc
    best_acc = max(current_acc, best_acc)
    checkpoint = {
        'best_acc': best_acc,
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    model_path = os.path.join('model', 'checkpoint.pth.tar')
    best_model_path = os.path.join('model', 'best_checkpoint.pth.tar')
    torch.save(checkpoint, model_path)
    if is_best:
        shutil.copy(model_path, best_model_path)